In [3]:
from IPython.display import display, HTML

# We embed the entire application (Logic + UI) into a single HTML string.
# This bypasses the 'ipywidgets' dependency completely.

html_code = """
<!DOCTYPE html>
<html>
<head>
<style>
    /* MODERN CSS STYLING */
    :root { --primary: #2c3e50; --accent: #3498db; --pivot: #d4edda; --target: #f8d7da; }
    body { font-family: 'Segoe UI', sans-serif; padding: 20px; background: #fff; }
    
    .app-container {
        border: 1px solid #ddd;
        border-radius: 8px;
        box-shadow: 0 4px 10px rgba(0,0,0,0.1);
        max-width: 800px;
        margin: auto;
        overflow: hidden;
    }
    
    .header { background: var(--primary); color: white; padding: 15px; text-align: center; }
    .content { padding: 20px; }
    
    /* INPUT GRID */
    .input-section { margin-bottom: 20px; text-align: center; }
    .matrix-grid {
        display: inline-grid;
        grid-template-columns: repeat(4, 70px);
        gap: 5px;
        padding: 10px;
        background: #f0f0f0;
        border-radius: 8px;
    }
    .matrix-grid input {
        width: 60px; padding: 8px; text-align: center; border: 1px solid #ccc; border-radius: 4px; font-weight: bold;
    }
    
    /* BUTTON & SLIDER */
    button {
        background: var(--accent); color: white; border: none; padding: 10px 20px; 
        border-radius: 4px; cursor: pointer; font-size: 16px; margin-top: 10px; width: 100%;
    }
    button:hover { background: #2980b9; }
    
    .slider-container { margin: 20px 0; display: none; } /* Hidden initially */
    input[type=range] { width: 100%; }
    
    /* VISUALIZATION */
    .step-display { 
        border: 1px solid #eee; padding: 15px; border-radius: 8px; min-height: 200px;
        display: none; /* Hidden initially */
    }
    .info-box { background: #e3f2fd; border-left: 5px solid #2196f3; padding: 10px; margin-bottom: 15px; }
    .math-text { font-family: 'Courier New', monospace; font-weight: bold; background: white; padding: 2px 6px; border-radius: 4px; }
    
    .vis-grid {
        display: grid; grid-template-columns: repeat(4, 1fr); gap: 4px; width: 300px; margin: auto;
    }
    .vis-cell {
        padding: 10px; text-align: center; border: 1px solid #ddd; border-radius: 4px; font-weight: bold;
    }
    .cell-pivot { background-color: var(--pivot); border-color: #28a745; }
    .cell-target { background-color: var(--target); border-color: #dc3545; }
    
    .cost-stats { display: flex; justify-content: space-around; margin-top: 15px; font-size: 13px; color: #555; background: #fafafa; padding: 10px; }
</style>
</head>
<body>

<div class="app-container">
    <div class="header"><h3>🎓 Gaussian Elimination Visualizer</h3></div>
    
    <div class="content">
        <div class="input-section">
            <p><strong>Edit System (Ax = b)</strong></p>
            <div class="matrix-grid" id="inputGrid"></div>
            <button onclick="runSolver()">▶ Solve & Visualize</button>
        </div>

        <div class="slider-container" id="sliderArea">
            <label>Timeline:</label>
            <input type="range" id="stepSlider" min="0" value="0" oninput="updateView(this.value)">
            <div style="text-align: right; font-size: 12px; color: #777;" id="stepCounter">Step 0</div>
        </div>

        <div class="step-display" id="outputArea">
            <div class="info-box">
                <div style="font-size: 12px; text-transform: uppercase; color: #555;">Current Action</div>
                <div id="msgBox" style="font-weight: bold; font-size: 16px; margin: 5px 0;"></div>
                <div id="mathBox" class="math-text"></div>
            </div>
            
            <div class="vis-grid" id="visGrid"></div>
            
            <div class="cost-stats">
                <span>Rows Swapped/Mixed: <b id="costOps">0</b></span>
                <span>Multiplications: <b id="costMult">0</b></span>
                <span>Additions: <b id="costAdd">0</b></span>
            </div>
        </div>
    </div>
</div>

<script>
    // ==========================================
    // 1. INITIALIZATION
    // ==========================================
    const defaultData = [
        2, 1, -1, 8,
        -3, -1, 2, -11,
        -2, 1, 2, -3
    ];
    
    // Generate Input Fields
    const grid = document.getElementById('inputGrid');
    defaultData.forEach((val, i) => {
        let inp = document.createElement('input');
        inp.type = "number";
        inp.value = val;
        inp.id = "c_" + i;
        grid.appendChild(inp);
    });

    // History Storage
    let history = [];

    // ==========================================
    // 2. SOLVER ENGINE (JavaScript Port)
    // ==========================================
    function runSolver() {
        // Read Inputs
        let mat = [];
        for(let r=0; r<3; r++){
            let row = [];
            for(let c=0; c<4; c++){
                let val = parseFloat(document.getElementById("c_" + (r*4+c)).value);
                row.push(val);
            }
            mat.push(row);
        }

        history = [];
        let cost = {ops:0, mult:0, add:0};
        
        // Helper to record state
        function record(m, msg, math, piv=null, tar=null) {
            history.push({
                mat: JSON.parse(JSON.stringify(m)), // Deep copy
                msg: msg,
                math: math,
                cost: {...cost},
                piv: piv,
                tar: tar
            });
        }

        record(mat, "Start: Initial Matrix", "Ax = b");

        // --- FORWARD ELIMINATION ---
        for(let i=0; i<3; i++) {
            let pivot = mat[i][i];
            
            // Check Pivot
            if(Math.abs(pivot) < 1e-10) {
                // Swap?
                let found = false;
                for(let k=i+1; k<3; k++){
                    if(Math.abs(mat[k][i]) > 1e-10) {
                        // Swap rows
                        let temp = mat[i]; mat[i] = mat[k]; mat[k] = temp;
                        record(mat, `Swapped Row ${i+1} ↔ Row ${k+1}`, `R${i+1} ↔ R${k+1}`, [i,i]);
                        pivot = mat[i][i];
                        found = true;
                        break;
                    }
                }
                if(!found) {
                    record(mat, "Singular Matrix", "FAILURE: Pivot is 0", [i,i]);
                    setupSlider(); return;
                }
            } else {
                 record(mat, `Checking Pivot (${i+1},${i+1})`, `Pivot = ${parseFloat(pivot.toPrecision(4))}`, [i,i]);
            }

            // Eliminate
            for(let j=i+1; j<3; j++) {
                let target = mat[j][i];
                if(Math.abs(target) > 1e-10) {
                    let m = target/pivot;
                    // Row Operation
                    for(let c=0; c<4; c++) {
                        mat[j][c] = mat[j][c] - m * mat[i][c];
                    }
                    cost.ops++; cost.mult+=4; cost.add+=4;
                    record(mat, `Eliminate (${j+1},${i+1})`, `R${j+1} ← R${j+1} - (${parseFloat(m.toPrecision(3))})R${i+1}`, [i,i], [j,i]);
                }
            }
        }

        // --- BACK SUB ---
        if(Math.abs(mat[2][2]) > 1e-10) {
            let x = [0,0,0];
            // Z
            x[2] = mat[2][3] / mat[2][2];
            record(mat, "Solving Z (Row 3)", `z = ${parseFloat(x[2].toPrecision(4))}`, [2,2]);
            
            // Y
            x[1] = (mat[1][3] - mat[1][2]*x[2]) / mat[1][1];
            record(mat, "Solving Y (Row 2)", `y = ${parseFloat(x[1].toPrecision(4))}`, [1,1]);

            // X
            x[0] = (mat[0][3] - mat[0][1]*x[1] - mat[0][2]*x[2]) / mat[0][0];
            record(mat, "Solving X (Row 1)", `x = ${parseFloat(x[0].toPrecision(4))}`, [0,0]);

            let res = `x=${parseFloat(x[0].toPrecision(3))}, y=${parseFloat(x[1].toPrecision(3))}, z=${parseFloat(x[2].toPrecision(3))}`;
            record(mat, "✅ Solution Found", res);
        }

        setupSlider();
    }

    // ==========================================
    // 3. UI UPDATER
    // ==========================================
    function setupSlider() {
        let slider = document.getElementById('stepSlider');
        slider.max = history.length - 1;
        slider.value = 0;
        document.getElementById('sliderArea').style.display = 'block';
        document.getElementById('outputArea').style.display = 'block';
        updateView(0);
    }

    function updateView(idx) {
        let step = history[idx];
        
        document.getElementById('stepCounter').innerText = `Step ${parseInt(idx) + 1} / ${history.length}`;
        document.getElementById('msgBox').innerText = step.msg;
        document.getElementById('mathBox').innerText = step.math;
        document.getElementById('costOps').innerText = step.cost.ops;
        document.getElementById('costMult').innerText = step.cost.mult;
        document.getElementById('costAdd').innerText = step.cost.add;

        // Render Grid
        let vis = document.getElementById('visGrid');
        vis.innerHTML = '';
        
        for(let r=0; r<3; r++) {
            for(let c=0; c<4; c++) {
                let div = document.createElement('div');
                div.className = 'vis-cell';
                let val = step.mat[r][c];
                div.innerText = (Math.abs(val) < 1e-10) ? "0" : parseFloat(val.toPrecision(4));
                
                // Color Logic
                if(step.piv && r===step.piv[0] && c===step.piv[1]) div.classList.add('cell-pivot');
                if(step.tar && r===step.tar[0] && c===step.tar[1]) div.classList.add('cell-target');
                if(c===3) div.style.borderLeft = "3px solid #333"; // Augment line
                
                vis.appendChild(div);
            }
        }
    }
</script>
</body>
</html>
"""

display(HTML(html_code))